### Lab 1a (Numerical Analysis, Fall 2015)

# <p style='text-align: center;'> Field testing integration algorithms </p>


**OBJECTIVES**: 

*  To get some experience designing, writing, and using Python functions.

*  To deepen your understanding of Python syntax.

*  To work through the kinds of numerical experiments one might conduct to test out an algorithm.


## Due by Wednesday, September 16

1.  This Jupyter notebook (lab1a.ipynb), as well as the  notebook I assign next Monday (lab1b.ipynb.)  Submit these files to your Dropbox.  I will check that your code runs correctly, that you've made an appropriate use of comments, and that any variable names you introduce are sensible and informative. 

2.  Hand in your lab notebook in class.  I will be looking for evidence of thoughtful engagement with the problems at hand, in accordance with the document "Reflections on Why and How to Keep a Laboratory Notebook" that I passed out in class.

## Introduction

In class we discussed several ways to produce numerical approximations of definite integrals, including the well-known Trapezoid and Midpoint rules.  Many of you have probably seen these algorithms before, most likely in a calculus class, and most of you will probably agree that they are geometrically intuitive.  (After all, the definite integral is defined as a Riemann sum, and in effect these algorithms just calculate Riemann sums explicitly.)  

What we have not done yet is engage in any serious discussion about how to evaluate these algorithms.  As we noted in class discussion, we have two basic questions:

1.  How fast?
2.  How accurate?

Speed will of course depend on a number of things, including the width of the spacing between sample points, the length of the interval of integration, and the complexity of each function evaluation.  Assessing the speed of an algorithm is thus a matter of making *comparative* claims, where the comparisons are made on a standard set of test problems, and compared against other standard algorithms.  (The process of making such comparative claims is generally called **benchmarking**.) Of course, we don't have any "standard" algorithms in our toolkit yet, so we'll simply need to compare these algorithms against one another, and perhaps use them as the basis of future comparisons.

Accuracy also depends on several things, including the stepsize and how smooth the integrand is.  Specifically, we expect that as the stepsize shrinks, the accuracy will increase, and as the function being integrated grows in complexity, the accuracy will decrease.  It will thus be important to assess our errors as a function of step size, and to quality any claims about accuracy with reference to the complexity of the test functions.   

In this lab, we're going to focus on the Trapezoid rule, the Right Hand Endpoint rule (a slight variation of the Midpoint rule), and Simpson's rule.  We haven't discussed Simpson's rule in class yet, but we will get to it this week, and many of you will remember it from calculus.  It turns out that the Midpoint rule, the Trapezoid rule, and Simpson's rule are all examples of what are known as *Newton-Cotes* formulas, of orders 0, 1, and 2, respectively. We'll derive the general Newton-Cotes formula in class and discuss the idea of "order", but for purposes of this lab, it suffices to think of Simpson's lab as the "next most complex" integration algorithm.  The purpose of this lab is to begin the process of assessing the performance of these algorithms, and hopefully to get you thinking about how you might choose and use one or the other of them in practice.


## In-class activities

The first order of business is to dedicate a page (or more) in your lab notebook to taking notes on Python itself.  These notes can involve specific commands, names of packages along with a short description, syntax conventions, etc.  The purpose to have a place to jot down things you discover in the course of this and subsequent labs.  

** Task 0:**

*Somewhere in your notebook, dedicate a page to writing down notes about Python.  Give the page a revealing title, something like ``Useful Python Commands", or ``Python Notes'', and add this page to your table of contents, so you know where to find it.  As you complete this and subsequent labs, keep notes about syntax, commands, packages, etc. that you find useful and think are worth keeping track of.*


The next order of business is to set up the workspace.  In this case, we import a few scientific libraries, and invoke some functionality that (hopefully) makes this code compatible with Python 3.



In [1]:
import numpy as np # numpy is Python's numerical library; always import it
import scipy as sp # scipy is a scientific library; it has a few functions we'll need
from __future__ import division, print_function # good defensive measure
import matplotlib as mpl # a big library with plotting functionality
import matplotlib.pyplot as plt # a subset of matplotlib with most of the useful tools
import time # we'll use this package to time how fast the code runs
%matplotlib inline 
%qtconsole 

/home/nbuser/anaconda3_410/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/home/nbuser/anaconda3_410/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Since some of you are new to Python, spending a little bit of time browsing the documentation for numpy, scipy, and matplotlib might be a good idea, as we'll be importing things from these packages regularly.  Here is a link to a page that serves as a gateway to the documentation files for all three packages (and more!):

**Numpy, Scipy, Matplotlib Docs**:  http://www.scipy.org/docs.html  

**Task 1 [optional]**  (Can be skipped if you're already pretty comfortable with numpy and numpy arrays.  Even if you do choose to do this, you might do it after you've completed the rest of the lab, so you can use classtime to do more interesting things.)

*Use the above link to navigate to the "Numpy User Guide" and skim the section called "What is Numpy?"  Then browse the section called "Array creation", focusing on mechanisms 1 and 2, and try creating a few arrays in your qtconsole (you can copy and paste the example commands, if you wish.)  If any of the above array creation commands are unfamiliar to you, you might jot them down.* 

Having imported the scientific functionality we need, we can set about implementing our three integration algorithms. I've written the Trapezoid method below as a function that takes four inputs (the name of the function to be integrated, the left and right hand endpoints, and the number of sample points) and produces one output (the approximate value of the integral.)  


In [2]:
def trapezoid(fname,a,b,n):
    '''
    function:  trapezoid(fname,a,b,n)

    Description:  integrates function 'fname' from a to b using the 
    composite trapezoid rule with n samples

    Inputs: 
        fname   name of function to be integrated
        a       left hand endpoint
        b       right hand endpoint
        n       number of intervals
        
    Outputs:
        I       a real number representing the value of the integral
    '''
    h = (b-a)/n
    x = np.linspace(a,b,n+1)
    I = h*(fname(x[0])/2+fname(x[n])/2+sum(fname(x[1:n])))
    return I

While it wasn't strictly necessary to implement Trapezoid as a function, you should get in the habit of functionlizing your code whenever possible:  functionalizing minimizes pollution of the global namespace, and facilitates code reuse.   

**Task 2**:

*The variable $x$ in the above function definition is a numpy array with n+1 elements evenly spaced between $a$ and $b$.  In your lab book, write down which element is in $x[n]$, and which elements are in $x[1:n]$.  Is there any overlap between these two sets of numbers?  If you're new to Python, pay particular attention here:  indexing conventions are occassionally counter-intuitive.*

**Task 3**:  

*Finish filling in the following code cell with the definition of the "Right Hand Endpoint" rule, given formally as*

$$
I = h \cdot \sum_1^n f(x_i),
$$ 
*where $h=(b-a)/n$, $x_i = a + i*h$, and $i=1, \cdots, n$.*

In [ ]:
def rhendpoint(fname,a,b,n):
    '''
    function:  rhendpoint(fname,a,b,n)

    Description:  integrates function 'fname' from a to b using the 
    composite right hand endpoint rule with n samples

    Inputs: 
        fname   name of function to be integrated
        a       left hand endpoint
        b       right hand endpoint
        n       number of intervals
    Outputs:
        I       a real number representing the value of the integral
    '''
    h = (b-a)/n
    x = 
    I = 
    return 

Finally, here is an implementation of Simpson's rule, given as

$$
I = \left( \frac{h}{3} \right) \cdot \left( f(x_0) + f(x_n) + 2 \sum_{i=2}^{n/2} f(x_{2i-2}) + 4 \sum_{i=1}^{n/2} f(x_{2i-1}) \right) \qquad (A)
$$

In [ ]:
def simpson(fname,a,b,n):
    '''
    function:  simpson(fname,a,b,n)

    Description:  integrates function 'fname' from a to b using the 
    composite Simpson's rule with n samples

    Inputs: 
        fname   name of function to be integrated
        a       left hand endpoint
        b       right hand endpoint
        n       number of intervals
    Outputs:
        I       a real number representing the value of the integral
    '''
    h = (b-a)/n
    x = np.linspace(a,b,n+1)
    I = (h/3)*(fname(x[0]) + fname(x[n]) + 2*sum(fname(x[range(2,n,2)])) + 4*sum(fname(x[range(1,n+1,2)])))
    return I

** Task 4 ** 

*In your notebook, explain how the command*

```4*sum(fname(x[range(1,n+1,2)])))```

*captures the last term in equation (A) above.  Check the documentation for  the Python command `range`, if necessary.  (Type `?range` at the command line.)*

----


We now need to define some test functions.  Here are a few:

$$
f(x) = (1+x) e^x
$$

$$
g(x) = x+\cos(50 \pi x/2)
$$

$$
h(x) = \frac{\log x}{\sqrt{x}}
$$

Note that the first is relatively smooth, the second is wildly oscillatory, and the third has a singularity at zero, but is still integrable.  This is a nice test suite--it covers a range of possible function behaviors, without including anything too exotic.  

We can implement these functions as follows:

In [3]:
def fun_f(x):
    return (1+x)*np.exp(x)

def fun_g(x):
    return x+np.cos(50*np.pi*x)

def fun_h(x):
    return 

**Task 5:**

*Finish defining the function `fun_h`.  Test that your function is correct by plugging in a few values and verifying that the output seems reasonable.*

We'll focus on the interval $[0,1]$.  In order to assess the accuracy of our approximations, we'll need to know the true values of the integrals of these functions.    For example, 

$$
\int_0^1 (1+x)e^x dx = \left. (e^x + xe^x - e^x) \right|^1_0 = e
$$

----

** Task 6**

*Figure out the value of $\int_0^1 g(x)dx$ and $\int_0^1 h(x)dx$.   Record your work in your lab book.*

----

Now let's run some tests.  We'll start with the exponential function, and use 20 sample points.  

In [4]:
# adjustable parameters
fname = fun_f
a = 0
b = 1
n = 20000
I_true    = np.exp(1)

# numerically approximate the integral using each method
t1 = time.time()
I1_approx = trapezoid(fname,a,b,n)
t2 = time.time()
print("Trapezoid:")
print("Relative error:", np.abs((I1_approx-I_true)/I_true))
print("Elapsed time:", t2-t1,"\n")

t1 = time.time()
I2_approx = rhendpoint(fname,a,b,n)
t2 = time.time()
print("RHEndpt:")
print("Relative error:", np.abs((I2_approx-I_true)/I_true))
print("Elapsed time:", t2-t1,"\n")

t1 = time.time()
I3_approx = simpson(fname,a,b,n)
t2 = time.time()
print("Simpson:")
print("Relative error:", np.abs((I3_approx-I_true)/I_true))
print("Elapsed time:", t2-t1,"\n")


Trapezoid:
Relative error: 4.71715323073e-10
Elapsed time: 0.006690263748168945 



NameError: name 'rhendpoint' is not defined

**Task 7**

*Run the above code snippet a few times, and take some notes on what you observe.  Things to think about:  are the results the same everytime you run the code?  Why might they not be?  Do you observane any tradeoffs between speed and accuracy?  Comment on what you obseve in your labbooks, and if this experiment suggests further questions, be sure to write them down.  **Extra:** If you have time, try playing around with the other functions, and seeing if you observe anything different.*

**Task 8:**

*Repeat the above, only now systematically change the number of sample points.  How big can you make n before these algorithms start to take a long time to converge?  Increase $n$ from 20 to 200, 2000, etc. until you notice a marked slowdown.  Make notes of what you observe.  Speculate as to why the slowdown occurs exactly where it occurs.*

Finally, let's systematically investigate how the error decreases as the number of sample points increases.  As we'll see in class this week, it's generally more convenient to examine the relation between $h$ (interval size) and the error, rather than $n$ (number of sample points) and the error.  Accordingly, the script below considers a range of $n$'s, converts each of them to an $h$, and plots a table that shows $h$, the error, and the error divided by $h^2$. 

In [ ]:
from tabulate import tabulate  # we'll use this to print out our output nicely

mvals        = 8      # the number of n values to investigate
ns           = [10**n for n in range(mvals)] # this is an efficient way to create a list
data         = []     # initialize an empty list

fname = fun_f
a = 0
b = 1

for i in range(mvals):
    I_approx = trapezoid(fname,a,b,ns[i])
    I_true = np.exp(1)
    h = (b-a)/ns[i]
    error = np.abs(I_approx-I_true)
    error_to_h_ratio = error/h**2
    data.append([h,error,error_to_h_ratio])
  

print(tabulate(data, headers=['h', 'error', 'error/h^2']))

**Task 9**:

*The first five or six rows of this table illustrate the same phenomena as described in our text, page XXX.  But at some point, the numbers in the third column deviate from seems to be their "steady state" value and start **increasing**.  Speculate as to why this might be so.  Discuss with your partner.  Record your thoughts in your lab book.*

**Task 10:**

*Redo the above table using Simpson's rule instead of the Trapezoid rule.  (It might be useful to copy and past the code to a new cell block, so you can compare outputs without writing over the old stuff.)  What do you observe?  Does the ratio "error/$h^2$" converge to a constant, as with the Trapezoid case?  Try figuring out what power of $h$ *does* generate this sort of convergence.*  